In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [12]:
team_stats = pd.read_csv('../../footy_data/england-championship-teams-2021-to-2022-stats.csv')
games = pd.read_csv('../../footy_data/england-championship-matches-2021-to-2022-stats.csv')

In [13]:
games_for_later = games
#I am saving this new variable so it will be the original df when I want to include games that are 'incomplete' later

In [14]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [15]:
games = games.loc[games['status'] == 'complete']
#We only want games that have been completed

In [16]:
team_stats['avg_home_xg_for_home_team'] = ""
team_stats['avg_away_xg_against_home_team'] = ""
team_stats['avg_xg_for_home_team'] = ""
team_stats['avg_xg_against_home_team'] = ""
team_stats['avg_home_xg_for_away_team'] = ""
team_stats['avg_away_xg_against_away_team'] = ""
team_stats['avg_xg_for_away_team'] = ""
team_stats['avg_xg_against_away_team'] = ""
#Creating average xG and xGA for each team myself because FootyStats is too lazy to put it correctly in the spreadsheet

In [17]:
# #CODE TO FIX XG AND XGA HERE, use numpy
# #average of home_team = common_name[0]xgfor and away_team = common_name[0] xgfor
# def get_avg_xg(team):
#     avg_for = games.loc[games['home_team_name'] == team_stats[team]
#     avg_for = avg_stoke.append(games.loc[games['away_team_name'] == team_stats['common_name'][0]])

In [18]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

<ipython-input-18-c9fa362d6826>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['home_xg'] = ""
<ipython-input-18-c9fa362d6826>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['away_xg'] = ""
<ipython-input-18-c9fa362d6826>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

Let's write a function to help fill xg in our games df

In [19]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

In [20]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [21]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [22]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 

#Reading in stats

<ipython-input-21-a42fdaf3b0fe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
<ipython-input-21-a42fdaf3b0fe>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
<ipython-input-21-a42fdaf3b0fe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

# Read in stats for new DF we'll use for testing

In [25]:
# test_prediction_games['home_xg'] = ""
# test_prediction_games['away_xg'] = ""
# test_prediction_games['home_xg_against'] = ""
# test_prediction_games['away_xg_against'] = ""
# test_prediction_games['home_total_corners'] = ""
# test_prediction_games['away_total_corners'] = ""

# # Creating new columns where our data will go

In [26]:
def fill_stats_new_test(team):
    test_prediction_games['home_xg'] = np.where(test_prediction_games['home_team_name'] == team, xg_stats[team], test_prediction_games['home_xg'])
    test_prediction_games['away_xg'] = np.where(test_prediction_games['away_team_name'] == team, xg_stats[team], test_prediction_games['away_xg'])
    test_prediction_games['home_xg_against'] = np.where(test_prediction_games['home_team_name'] == team, xga_stats[team], test_prediction_games['home_xg_against'])
    test_prediction_games['away_xg_against'] = np.where(test_prediction_games['away_team_name'] == team, xga_stats[team], test_prediction_games['away_xg_against'])
    test_prediction_games['home_total_corners'] = np.where(test_prediction_games['home_team_name'] == team, corner_stats[team], test_prediction_games['home_total_corners'])
    test_prediction_games['away_total_corners'] = np.where(test_prediction_games['away_team_name'] == team, corner_stats[team], test_prediction_games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [27]:
fill_stats_new_test('Fulham')
fill_stats_new_test('Blackburn Rovers')
fill_stats_new_test('West Bromwich Albion')
fill_stats_new_test('Queens Park Rangers')
fill_stats_new_test('AFC Bournemouth')
fill_stats_new_test('Middlesbrough')
fill_stats_new_test('Barnsley')
fill_stats_new_test('Millwall')
fill_stats_new_test('Sheffield United')
fill_stats_new_test('Reading')
fill_stats_new_test('Cardiff City')
fill_stats_new_test('Nottingham Forest')
fill_stats_new_test('Hull City')
fill_stats_new_test('Blackpool')
fill_stats_new_test('Luton Town')
fill_stats_new_test('Stoke City')
fill_stats_new_test('Swansea City')
fill_stats_new_test('Derby County')
fill_stats_new_test('Huddersfield Town')
fill_stats_new_test('Preston North End')
fill_stats_new_test('Coventry City')
fill_stats_new_test('Peterborough United')
fill_stats_new_test('Birmingham City')
fill_stats_new_test('Bristol City')

NameError: name 'test_prediction_games' is not defined

# Some more EDA

In [28]:
games.corr()['over/under'].sort_values(ascending=False)[0:20]

over/under                        1.000000
total_goal_count                  0.813885
away_team_goal_count              0.564549
home_team_goal_count              0.564415
total_goals_at_half_time          0.431120
home_team_shots_on_target         0.338370
home_team_goal_count_half_time    0.309042
away_team_goal_count_half_time    0.281679
away_team_shots_on_target         0.280491
odds_btts_no                      0.219733
team_a_xg                         0.211337
home_ppg                          0.199479
home_team_shots                   0.169254
home_team_fouls                   0.140905
Pre-Match PPG (Home)              0.109827
odds_ft_draw                      0.098503
team_b_xg                         0.087455
odds_ft_away_team_win             0.084549
away_team_red_cards               0.067356
away_ppg                          0.052869
Name: over/under, dtype: float64

# Now let's do some modeling

This will be a 'normal' model in which we use train_test_split to divide up the data

In [16]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X = games[features]
y = games['over/under']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [18]:
logreg = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()

In [19]:
logreg.fit(X_train, y_train)

LogisticRegression()

In [20]:
preds = logreg.predict(X_test)

In [21]:
accuracy_score(y_test, preds)

0.32

In [22]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [23]:
preds = knn.predict(X_test)

In [24]:
accuracy_score(y_test, preds)

0.4

In [25]:
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [26]:
preds = dt.predict(X_test)

In [27]:
accuracy_score(y_test, preds)

0.44

In [28]:
#decision trees

In [29]:
y_test

44     1
47     0
4      0
55     0
26     0
64     0
73     0
10     0
40     1
108    1
18     1
62     1
11     1
36     0
90     1
118    0
110    0
0      1
89     0
104    0
65     0
45     0
31     0
70     0
42     0
Name: over/under, dtype: int32

Test the test set while it is trained on the original train set?

In [30]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X = test_prediction_games[features]
y = test_prediction_games['over/under']

NameError: name 'test_prediction_games' is not defined

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.9, random_state = 42)

In [32]:
games[0:5]['home_team_name']

0     AFC Bournemouth
1    Blackburn Rovers
2        Bristol City
3        Cardiff City
4        Derby County
Name: home_team_name, dtype: object

# Trying to set the train and test sets myself without train_test_split

# Predicting Over/Under 2.5

In [33]:
# games = pd.read_csv('../footy_data/championship_games_2022.csv')
#Going to load this in again because I need the full dataset including incomplete games -- the ones I want to predict

In [34]:
games.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_btts_yes,odds_btts_no,stadium_name,over/under,home_xg,away_xg,home_xg_against,away_xg_against,home_total_corners,away_total_corners
0,1628275500,Aug 06 2021 - 6:45pm,complete,NaN,AFC Bournemouth,West Bromwich Albion,Dean Whitestone,1,0.0,0.0,...,1.80,2.05,Vitality Stadium (Bournemouth- Dorset),1,1.16,1.97,1.04,0.77,4.6,6.4
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Blackburn Rovers,Swansea City,David Webb,1,0.0,0.0,...,1.95,1.87,Ewood Park (Blackburn- Lancashire),1,1.14,0.78,1.43,1.21,3.3,4.2
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Bristol City,Blackpool,Andy Davies,1,0.0,0.0,...,1.95,1.91,Ashton Gate (Bristol),0,1.15,1,1.47,1.48,3.7,4.3
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Cardiff City,Barnsley,Steve Martin,1,0.0,0.0,...,1.95,1.91,Cardiff City Stadium (Cardiff (Caerdydd)),0,1.23,0.78,1.19,1.33,3.9,3.8
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Derby County,Huddersfield Town,Geoff Eltringham,1,0.0,0.0,...,1.95,1.87,Pride Park Stadium (Derby),0,0.93,0.98,1.57,1.26,4.3,4.4


In [35]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X_train = games[0:72][features]
X_test = games[72:84][features]
y_train = games[0:72]['over/under']
y_test = games[72:84]['over/under']

In [36]:
X_test

,home_xg,away_xg,home_xg_against,away_xg_against
72,1.16,1.56,1.04,1.34
73,1.14,0.99,1.43,1.33
74,1,0.98,1.48,1.26
75,1.58,1.11,1.02,1.2
76,1.2,0.63,1.6,1.46
77,1.97,0.93,0.77,1.57
78,1.28,2,1.04,1.01
79,1.15,1.43,1.47,1.23
80,1.52,1.23,1.28,1.19
81,0.95,1.23,1.26,1.19


In [37]:
logreg.fit(X_train, y_train)

LogisticRegression()

In [38]:
preds = logreg.predict(X_test)

In [39]:
accuracy_score(y_test, preds)

0.4166666666666667

In [40]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [41]:
preds = knn.predict(X_test)

In [42]:
accuracy_score(y_test, preds)

0.5

In [43]:
preds

array([1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1])

Next need to add something to track accuracy from week to week